In [16]:
%matplotlib inline
import re
import random

import pandas as pd
import matplotlib.pyplot as plt

from mecab import MeCab

from sklearn.model_selection import train_test_split

### DATA

1. 텍스트 데이터 분석 
2. 에러 데이터 생성

In [2]:
df = pd.read_csv('../../data/raw/colloquial_ko.csv')

In [3]:
df.head()

,tgt
0,"그 말을 들으니 기쁘고, 저희와 거래하는 것을 고려해 주셨으면 합니다."
1,"확실히 생각하고 있습니다만, 몇 가지 여쭤보고 싶은 게 있어요."
2,"그들에게 말하면, 저희가 그 아이들을 돌볼 것입니다."
3,"좋아요, 수영은 어떤가요?"
4,현재 두 개의 수영장이 있습니다.


In [4]:
mecab = MeCab()

In [5]:
df['morphs_ko'] = df['tgt'].apply(lambda x: mecab.morphs(str(x)))

In [6]:
df['morphs_length'] = df['morphs_ko'].apply(lambda x: len(x))

In [7]:
df.describe()

,morphs_length
count,1.260280e+06
mean,1.391222e+01
std,5.386720e+00
min,1.000000e+00
25%,1.000000e+01
50%,1.300000e+01
75%,1.700000e+01
max,7.800000e+01


In [15]:
plt.

0          21
1          19
2          17
3           8
4           9
           ..
1260275     9
1260276    13
1260277     9
1260278    18
1260279    17
Name: morphs_length, Length: 1260280, dtype: int64

- 텍스트 데이터 형태소 분석 결과 : 대부분 10개(전체 데이터의 약 25%) ~ 17개 (전체 데이터의 약 75%)의 형태소로 이루어져있음.

- 학습 및 테스트 데이터 약 5만개 정도 샘플링 하되 위의 형태소 개수로 이루어진 데이터셋 구축 예정. 

In [8]:
df_idx = [i for i in range(len(df)) if 10 <= df['morphs_length'][i] < 18]

In [9]:
# 약 50만개의 데이터가 걸러짐
print(len(df))
print(len(df_idx))
df2 = df.loc[df_idx]
df2.reset_index(inplace=True, drop=True)

1260280
729137


In [10]:
def random_sampling(df, random_seed:int, sample_num:int, data:str):
    random.seed(random_seed)
    if sample_num > len(df):
        random_sample_index = random.sample(range(len(df)), len(df))
    else:
        random_sample_index = random.sample(range(len(df)), sample_num)

    if data == 'train':
        result_df = df.drop(random_sample_index)
        result_df.reset_index(inplace=True, drop=True)
        return result_df
    elif data == 'test':
        result_df = df.loc[random_sample_index]
        result_df.reset_index(inplace=True, drop=True)
        return result_df 

In [11]:
sample = random_sampling(df2, random_seed=123, sample_num=50000, data = 'test')

#### Subword 분절

- 서브워드 분절은 기본적으로 '단어는 의미를 가진 더 작은 서브워드들의 조합으로 이루어진다'는 가정하에 적용되는 알고리즘

- 적절한 서브워드를 발견하여 해당 단위로 쪼개어주면 어휘수를 줄일 수 있고 희소성을 효과적으로 줄일 수 있다.

- 신조어나 오타와 같은 UNK에 대해 서브워드 단위나 문자 단위로 쪼개줌으로써 기존 훈련 데이터에서 보았던 토큰들의 조합으로 만들 수 있다.

- 자연어 전처리에 필수 절차